# Parallelism


In [1]:
%load_ext memory_profiler
%memit import src as xc    
%memit import xarray as xr  
from dask.distributed import Client 
client = Client(n_workers=8, threads_per_worker=2)

%memit obs = xr.open_dataset('test_data/cpc_jaso_asia_full.nc', decode_times=False, chunks='auto')
%memit cola = xr.open_dataset('test_data/cola_jaso_asia_full.nc', decode_times=False, chunks='auto')
%memit cansipsv2 = xr.open_dataset('test_data/cansipsv2_jaso_asia_full.nc', decode_times=False, chunks='auto')
%memit cfsv2 = xr.open_dataset('test_data/cfsv2_jaso_asia_full.nc', decode_times=False, chunks='auto')

%memit Y = obs.expand_dims({'M':[0]})
%memit Y = Y * 3

%memit X = xr.concat([cola, cfsv2, cansipsv2], 'M').assign_coords({'M':[0,1,2]})
%memit X_TRAIN = X.isel(S=slice(None, -7))
%memit X_TEST = X.isel(S=slice(-7, None))

peak memory: 255.30 MiB, increment: 210.28 MiB
peak memory: 255.32 MiB, increment: 0.01 MiB
peak memory: 267.63 MiB, increment: 2.34 MiB
peak memory: 268.01 MiB, increment: 0.38 MiB
peak memory: 268.69 MiB, increment: 0.68 MiB
peak memory: 269.28 MiB, increment: 0.59 MiB
peak memory: 269.30 MiB, increment: 0.02 MiB
peak memory: 269.46 MiB, increment: 0.15 MiB
peak memory: 269.77 MiB, increment: 0.31 MiB
peak memory: 269.83 MiB, increment: 0.07 MiB
peak memory: 269.86 MiB, increment: 0.03 MiB


In [2]:
%%time
%memit pmlp = xc.ParallelMultiLayerPerceptron(ND=1, max_iter=10, client=client)
%memit pmlp.fit(X_TRAIN.prec, Y.prate, x_sample_dim='S', verbose=True, lat_chunks=4, lon_chunks=4)


peak memory: 269.99 MiB, increment: 0.11 MiB
<Client: 'tcp://127.0.0.1:49757' processes=8 threads=16, memory=16.00 GiB>
peak memory: 431.79 MiB, increment: 161.79 MiB
CPU times: user 6.5 s, sys: 1.08 s, total: 7.58 s
Wall time: 40.9 s


In [3]:
%%time
%memit parallel_results =  pmlp.predict(X_TRAIN.prec, x_sample_dim='S', verbose=2, lat_chunks=4, lon_chunks=4)

Predicting MLPRegressor: 2021-09-15 12:53:00.126786 [*************************]
peak memory: 414.27 MiB, increment: 0.03 MiB
CPU times: user 3.86 s, sys: 741 ms, total: 4.6 s
Wall time: 5.26 s


In [4]:
sizeof(X_TRAIN.prec)
#.isel(M=0, S=0).plot()

NameError: name 'sizeof' is not defined